# LLY-DML: Quantum Circuit Optimization Tutorial

<div style="text-align: center; padding: 10px; margin: 10px;">
    <h3>LILY Quantum Machine Learning - Differentiable Machine Learning</h3>
    <p style="font-style: italic;">Ein Tutorial zur Quantenoptimierung mit L-Gates</p>
</div>

---

## Einführung

Das LLY-DML-Framework ermöglicht die Optimierung von Quantum Circuits mit einem speziellen Fokus auf L-Gate-Strukturen. Dieses Notebook führt Sie durch den Prozess der:

1. Erstellung eines Quantum Circuits mit L-Gates
2. Parametrisierung des Circuits mit trainierbaren Parametern
3. Optimierung dieser Parameter mit dem Adam-Algorithmus
4. Analyse und Visualisierung der Optimierungsergebnisse

## Was sind L-Gates?

Die L-Gate-Struktur ist eine spezielle Sequenz von Quantengates, die für Quantenmaschinelles Lernen optimiert wurde:

```
TP0 → IP0 → H → TP1 → IP1 → H → TP2 → IP2
```

Wobei:
- **TP** = Trainingsphase (P-Gate mit trainierbaren Parametern)
- **IP** = Inputphase (P-Gate mit festen Inputparametern)
- **H** = Hadamard-Gate (festes Gate ohne Parameter)

Diese Struktur wird für jeden Qubit und in der festgelegten Tiefe (Depth) wiederholt.

## 1. Setup und Bibliotheken importieren

In [ ]:
# Abhängigkeiten importieren
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import logging

# Importiere Qiskit für Visualisierung
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer

# Konfiguriere Pfade für Module-Imports
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(os.path.dirname(notebook_dir))
if project_root not in sys.path:
    sys.path.append(project_root)

# Konfiguriere Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("DML-Tutorial")

# Setze Matplotlib-Stil
plt.style.use('ggplot')
sns.set(style="whitegrid")

logger.info("Setup abgeschlossen. LLY-DML Tutorial bereit.")

# Zeige die Verzeichnisstruktur
print(f"Notebooks-Verzeichnis: {notebook_dir}")
print(f"Projekt-Root: {project_root}")

## 2. Importiere LLY-DML Module

Wir importieren die für das Tutorial benötigten Module aus dem LLY-DML Framework.

In [ ]:
# Importiere Circuit und Optimizer aus dem DML-Framework
from dml.module.src.circuit import Circuit
from dml.module.optimizers.adam_optimizer import AdamOptimizer

# Überprüfe, ob die Importe erfolgreich waren
print(f"Circuit-Klasse erfolgreich importiert: {Circuit.__name__}")
print(f"Adam-Optimizer erfolgreich importiert: {AdamOptimizer.__name__}")

## 3. Erstellen eines Quantum Circuits mit L-Gates

Hier erstellen wir einen Circuit mit:
- 5 Qubits
- Tiefe (Depth) 3 (d.h. 3 L-Gates pro Qubit)

Jedes L-Gate besteht aus mehreren Phasen-Gates und Hadamard-Gates in einer bestimmten Reihenfolge.

In [ ]:
# Parameter für den Circuit
qubits = 5
depth = 3
shots = 1024

# Erstelle einen Quantum Circuit
circuit = Circuit(qubits=qubits, depth=depth, shots=shots)
logger.info(f"Quantum Circuit erstellt mit {qubits} Qubits und Tiefe {depth}")

# Zeige Eigenschaften des Circuits
print(f"Circuit-Eigenschaften:")
print(f"Qubits: {circuit.qubits}")
print(f"Tiefe: {circuit.depth}")
print(f"Shots: {circuit.shots}")

# Visualisiere den Circuit
fig = circuit.plot_circuit()
plt.figure(figsize=(16, 6))
plt.title("Quantum Circuit mit L-Gates-Struktur")
plt.tight_layout()
plt.show()

## 4. Darstellung der L-Gate-Struktur

Lass uns die L-Gate-Struktur genauer betrachten. Jedes L-Gate folgt diesem Muster:

1. TP0 (Trainingsphase 0) - P-Gate mit trainierbarem Parameter
2. IP0 (Inputphase 0) - P-Gate mit Inputparameter
3. H - Hadamard-Gate
4. TP1 (Trainingsphase 1) - P-Gate mit trainierbarem Parameter
5. IP1 (Inputphase 1) - P-Gate mit Inputparameter
6. H - Hadamard-Gate
7. TP2 (Trainingsphase 2) - P-Gate mit trainierbarem Parameter
8. IP2 (Inputphase 2) - P-Gate mit Inputparameter

Wir visualisieren dies mit einem einfachen Qubit für bessere Übersichtlichkeit.

In [ ]:
# Erstelle einen einfachen L-Gate-Circuit für die Visualisierung
l_gate_circuit = QuantumCircuit(1)

# Füge ein L-Gate hinzu
# TP0 → IP0 → H → TP1 → IP1 → H → TP2 → IP2
l_gate_circuit.p(0.1, 0)  # TP0 - Trainingsphase 0
l_gate_circuit.p(0.2, 0)  # IP0 - Inputphase 0
l_gate_circuit.h(0)       # H   - Hadamard
l_gate_circuit.p(0.3, 0)  # TP1 - Trainingsphase 1
l_gate_circuit.p(0.4, 0)  # IP1 - Inputphase 1
l_gate_circuit.h(0)       # H   - Hadamard
l_gate_circuit.p(0.5, 0)  # TP2 - Trainingsphase 2
l_gate_circuit.p(0.6, 0)  # IP2 - Inputphase 2

# Visualisiere den L-Gate-Circuit
plt.figure(figsize=(14, 3))
circuit_drawer(l_gate_circuit, output='mpl')
plt.title("Einzelnes L-Gate")
plt.show()

# Erläuterung des L-Gates
print("L-Gate-Struktur:")
print("1. TP0 (P-Gate): Trainingsphase 0 - trainierbar")
print("2. IP0 (P-Gate): Inputphase 0 - fester Input")
print("3. H (Hadamard): Superposition erzeugen")
print("4. TP1 (P-Gate): Trainingsphase 1 - trainierbar")
print("5. IP1 (P-Gate): Inputphase 1 - fester Input")
print("6. H (Hadamard): Superposition erzeugen")
print("7. TP2 (P-Gate): Trainingsphase 2 - trainierbar")
print("8. IP2 (P-Gate): Inputphase 2 - fester Input")

## 5. Erstellen von Trainings- und Eingabematrizen

Für die Optimierung benötigen wir zwei Arten von Parametern:
1. **Trainingsparameter**: Diese werden durch den Optimierer angepasst
2. **Eingabeparameter**: Diese bleiben während der Optimierung konstant

Beide werden in spezialisierten Matrizen gespeichert.

In [ ]:
# Erstelle eine Trainingsmatrix
# Die Form der Matrix ist [qubits × depth × 3] - für jede TP-Phase eines jeden L-Gates
training_matrix = np.random.uniform(0, np.pi/4, (qubits, depth, 3))  # Initialisierung mit kleinen Werten
print(f"Trainingsmatrix-Form: {training_matrix.shape}")
print("Trainingsmatrix (erste Zeile):\n", training_matrix[0])

# Erstelle mehrere Eingabematrizen für verschiedene Klassifikationsaufgaben
num_input_matrices = 4
input_matrices = []
input_matrix_names = []

for i in range(num_input_matrices):
    # Verwende unterschiedliche Bereiche, um die Matrizen zu unterscheiden
    matrix = np.random.uniform(i*0.5, i*0.5 + 2*np.pi, (qubits, depth, 3))
    input_matrices.append(matrix)
    input_matrix_names.append(f"Matrix_{i+1}")
    
print(f"\nErzeugt: {num_input_matrices} Eingabematrizen")

# Visualisiere die Trainings- und Eingabematrizen
fig, axes = plt.subplots(1, min(num_input_matrices + 1, 5), figsize=(18, 5))

# Zeige die Trainingsmatrix
training_flat = training_matrix.reshape(qubits, -1)
sns.heatmap(training_flat, ax=axes[0], cmap="viridis", vmin=0, vmax=2*np.pi)
axes[0].set_title("Trainingsmatrix")
axes[0].set_xlabel("Parameter")
axes[0].set_ylabel("Qubit")

# Zeige die Eingabematrizen (bis zu 4)
for i in range(min(num_input_matrices, 4)):
    input_flat = input_matrices[i].reshape(qubits, -1)
    sns.heatmap(input_flat, ax=axes[i+1], cmap="plasma", vmin=0, vmax=2*np.pi)
    axes[i+1].set_title(f"Eingabematrix {i+1}")
    axes[i+1].set_xlabel("Parameter")
    axes[i+1].set_ylabel("Qubit")

plt.tight_layout()
plt.show()

## 6. Definition von Zielzuständen für jede Eingabematrix

Für die Optimierung müssen wir jeder Eingabematrix einen gewünschten Zielzustand zuweisen. In einem Klassifikationsszenario möchten wir, dass der Circuit für verschiedene Eingabematrizen unterschiedliche Zustände am Ende der Messung produziert.

In [ ]:
# Wir erstellen für jede Eingabematrix einen eindeutigen Zielzustand
target_states = {}

# Binärzahlen als Zielzustände
for i in range(num_input_matrices):
    # Erzeuge eine binäre Darstellung mit der richtigen Anzahl von Bits (qubits)
    binary = format(i+1, f'0{qubits}b')
    # Stelle sicher, dass der String die richtige Länge hat (gleich qubits)
    target_states[input_matrix_names[i]] = binary[-qubits:]

print("Zugewiesene Zielzustände:")
for matrix_name, state in target_states.items():
    print(f"{matrix_name}: '{state}'")

## 7. Ausführen eines Quantum Circuits und Analyse der Messungen

Bevor wir mit der Optimierung beginnen, führen wir einen Circuit mit einer Eingabematrix aus, um zu verstehen, wie die Messungen aussehen und wie weit wir vom Zielzustand entfernt sind.

In [ ]:
# Wir wählen die erste Eingabematrix
selected_matrix_idx = 0
input_matrix = input_matrices[selected_matrix_idx]
matrix_name = input_matrix_names[selected_matrix_idx]
target_state = target_states[matrix_name]

# Erstelle einen neuen Circuit mit den angegebenen Parametern
test_circuit = Circuit(qubits, depth, training_phases=training_matrix.tolist(), 
                       activation_phases=input_matrix.tolist(), shots=1024)

# Führe den Circuit aus
test_circuit.run()
counts = test_circuit.get_counts()

# Konvertiere die Counts in Wahrscheinlichkeiten
total_shots = sum(counts.values())
probabilities = {state: count/total_shots for state, count in counts.items()}

# Sortiere die Zustände nach absteigender Wahrscheinlichkeit
sorted_states = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

# Zeige die Ergebnisse
print(f"Ausführung des Circuits mit {matrix_name}")
print(f"Zielzustand: '{target_state}'")
print("\nTop 5 gemessene Zustände:")
for i, (state, prob) in enumerate(sorted_states[:5]):
    print(f"{i+1}. Zustand '{state}' mit Wahrscheinlichkeit {prob:.4f} ")
    if state == target_state:
        print("   ✓ Entspricht dem Zielzustand!")
    else:
        print("   ✗ Entspricht nicht dem Zielzustand")
        
# Berechne die Wahrscheinlichkeit des Zielzustands
target_probability = probabilities.get(target_state, 0)
print(f"\nWahrscheinlichkeit des Zielzustands '{target_state}': {target_probability:.4f}")

# Visualisiere die Wahrscheinlichkeiten
plt.figure(figsize=(14, 6))

# Limitiere auf die Top 10 Zustände für bessere Übersichtlichkeit
top_states = sorted_states[:10]
states = [s[0] for s in top_states]
probs = [s[1] for s in top_states]
colors = ['green' if s == target_state else 'blue' for s in states]

plt.bar(states, probs, color=colors)
plt.title(f"Top 10 Zustände für {matrix_name} (Zielzustand: {target_state})")
plt.xlabel("Quantum Zustand")
plt.ylabel("Wahrscheinlichkeit")
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Implementierung des Optimierungsprozesses

Jetzt implementieren wir den Optimierungsprozess, der die Trainingsmatrix optimiert, um die Wahrscheinlichkeit der Zielzustände für jede Eingabematrix zu maximieren.

Wir verwenden den Adam-Optimierer, der aus dem LLY-DML Framework importiert wurde.

In [ ]:
def optimize_for_target_state(input_matrix, training_matrix, target_state, iterations=20):
    """
    Optimiert die Trainingsmatrix für eine bestimmte Eingabematrix und einen Zielzustand.
    
    Args:
        input_matrix: Die Eingabematrix für den Circuit
        training_matrix: Die initiale Trainingsmatrix
        target_state: Der gewünschte Zielzustand als binärer String
        iterations: Anzahl der Trainingsiterationen
        
    Returns:
        tuple: (optimierte Trainingsmatrix, Trainingsgeschichte)
    """
    # Konfigurationsdaten für den Optimizer vorbereiten
    optimizer_data = {
        "qubits": qubits,
        "depth": depth,
        "learning_rate": 0.01
    }
    
    # Adam-Optimizer initialisieren
    # Der Optimizer arbeitet mit dem ersten Bit des Zielzustands
    optimizer = AdamOptimizer(
        data=optimizer_data,
        training_matrix=training_matrix.flatten().tolist(),
        target_state=target_state[0],  # Wir verwenden nur das erste Bit für den Optimizer
        learning_rate=0.01,
        max_iterations=iterations
    )
    
    # Stelle sicher, dass das tuning_parameters-Attribut existiert
    if not hasattr(optimizer, 'tuning_parameters'):
        optimizer.tuning_parameters = np.array(training_matrix.flatten().tolist())
    
    # Trainingsgeschichte initialisieren
    history = []
    current_training_matrix = training_matrix.copy()
    
    # Circuit für die Optimierung erstellen
    circuit = Circuit(qubits, depth)
    
    # Training über mehrere Iterationen durchführen
    for iteration in range(iterations):
        # Aktuelle Trainings- und Eingabematrix in den Circuit einfügen
        circuit.place_input_parameters(input_matrix)
        circuit.set_train_parameters(current_training_matrix)
        circuit.place_train_matrix()
        
        # Messung hinzufügen und Circuit ausführen
        circuit.place_measurement()
        counts = circuit.execute_circuit(shots=1024)
        
        # Optimierungsschritt durchführen
        try:
            optimized_params, opt_steps = optimizer.optimize()
            
            # Reshape der optimierten Parameter zurück in die ursprüngliche Form
            if len(optimized_params) == qubits * depth * 3:
                reshaped_params = np.array(optimized_params).reshape(qubits, depth, 3)
                current_training_matrix = reshaped_params
            else:
                logger.warning(f"Optimierte Parameter haben falsche Größe: {len(optimized_params)}")
                # Verwende die vorherige Matrix, wenn die neue falsch ist
        except Exception as e:
            logger.error(f"Fehler bei der Optimierung: {e}")
            continue
            
        # Wahrscheinlichkeit des Zielzustands berechnen
        probabilities = circuit.get_state_probabilities(counts)
        target_prob = probabilities.get(target_state, 0.0)
        
        # Verlust aus dem letzten Optimierungsschritt
        loss = opt_steps[-1]["loss"] if opt_steps else 1.0
        
        # Trainingsschritt in der Historie speichern
        history.append({
            "iteration": iteration + 1,
            "target_probability": target_prob,
            "loss": loss
        })
        
        # Status ausgeben
        logger.info(f"Iteration {iteration+1}/{iterations}, "
                    f"Zielwahrscheinlichkeit: {target_prob:.4f}, Loss: {loss:.4f}")
    
    return current_training_matrix, history

## 9. Durchführung der Optimierung für alle Eingabematrizen

Jetzt führen wir die Optimierung für jede Eingabematrix durch und zeichnen den Fortschritt auf.

In [ ]:
# Parameter für die Optimierung
training_iterations = 20

# Ergebnisse speichern
optimization_results = {
    "matrices": [],
    "initial_states": {},
    "final_states": {},
    "training_history": {}
}

# Initialisiere die gemeinsame Trainingsmatrix
current_training_matrix = training_matrix.copy()

# Optimiere für jede Eingabematrix
for i, (matrix, name) in enumerate(zip(input_matrices, input_matrix_names)):
    target_state = target_states[name]
    
    print(f"\n{'='*50}")
    print(f"Optimierung für {name} mit Zielzustand '{target_state}'")
    print(f"{'='*50}")
    
    # Führe die Optimierung durch
    optimized_matrix, history = optimize_for_target_state(
        input_matrix=matrix,
        training_matrix=current_training_matrix,
        target_state=target_state,
        iterations=training_iterations
    )
    
    # Aktualisiere die Trainingsmatrix für die nächste Eingabematrix
    current_training_matrix = optimized_matrix
    
    # Speichere die Ergebnisse
    optimization_results["matrices"].append(matrix.tolist())
    optimization_results["initial_states"][i] = {
        "name": name,
        "state": target_state
    }
    
    # Berechne die finale Wahrscheinlichkeit
    final_prob = history[-1]["target_probability"] if history else 0.0
    optimization_results["final_states"][i] = {
        "state": target_state,
        "probability": final_prob,
        "loss": history[-1]["loss"] if history else 1.0
    }
    
    optimization_results["training_history"][i] = history
    
    print(f"Optimierung abgeschlossen. Finale Wahrscheinlichkeit für '{target_state}': {final_prob:.4f}")

# Speichere die Ergebnisse in einer JSON-Datei
results_dir = os.path.join(notebook_dir, "results")
os.makedirs(results_dir, exist_ok=True)

results_file = os.path.join(results_dir, f"optimization_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
with open(results_file, 'w') as f:
    json.dump(optimization_results, f, indent=2)

print(f"\nOptimierungsergebnisse gespeichert in: {results_file}")

## 10. Visualisierung der Optimierungsergebnisse

Wir visualisieren den Optimierungsfortschritt und die erreichten Wahrscheinlichkeiten für jede Eingabematrix.

In [ ]:
# Visualisiere den Trainingsfortschritt
plt.figure(figsize=(14, 6))

for matrix_idx, history in optimization_results["training_history"].items():
    iterations = [entry["iteration"] for entry in history]
    probabilities = [entry["target_probability"] for entry in history]
    
    matrix_name = optimization_results["initial_states"][matrix_idx]["name"]
    target_state = optimization_results["initial_states"][matrix_idx]["state"]
    
    plt.plot(iterations, probabilities, marker='o', label=f"{matrix_name} → '{target_state}'")

plt.title("Trainingsfortschritt - Wahrscheinlichkeit des Zielzustands")
plt.xlabel("Iteration")
plt.ylabel("Wahrscheinlichkeit")
plt.grid(True, alpha=0.3)
plt.legend()
plt.ylim(0, 1.0)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, "training_progress.png"), dpi=150)
plt.show()

# Visualisiere die finalen Wahrscheinlichkeiten
plt.figure(figsize=(12, 6))

matrix_indices = list(optimization_results["final_states"].keys())
matrix_names = [optimization_results["initial_states"][idx]["name"] for idx in matrix_indices]
target_states = [optimization_results["initial_states"][idx]["state"] for idx in matrix_indices]
final_probs = [optimization_results["final_states"][idx]["probability"] for idx in matrix_indices]

x = np.arange(len(matrix_indices))
plt.bar(x, final_probs, color='coral')
plt.title("Finale Wahrscheinlichkeiten nach der Optimierung")
plt.xlabel("Eingabematrix")
plt.ylabel("Wahrscheinlichkeit des Zielzustands")
plt.xticks(x, [f"{name}\n'{state}'" for name, state in zip(matrix_names, target_states)])
plt.grid(True, axis='y', alpha=0.3)
plt.ylim(0, 1.0)

# Füge Werte über den Balken hinzu
for i, prob in enumerate(final_probs):
    plt.text(i, prob + 0.02, f"{prob:.3f}", ha='center')

plt.tight_layout()
plt.savefig(os.path.join(results_dir, "final_probabilities.png"), dpi=150)
plt.show()

## 11. Analyse der optimierten Trainingsmatrix

Lass uns die optimierte Trainingsmatrix im Vergleich zur ursprünglichen analysieren.

In [ ]:
# Vergleiche Original- und optimierte Trainingsmatrix
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Original Trainingsmatrix
original_flat = training_matrix.reshape(qubits, -1)
sns.heatmap(original_flat, ax=axes[0], cmap="viridis", vmin=0, vmax=2*np.pi)
axes[0].set_title("Originale Trainingsmatrix")
axes[0].set_xlabel("Parameter")
axes[0].set_ylabel("Qubit")

# Optimierte Trainingsmatrix
optimized_flat = current_training_matrix.reshape(qubits, -1)
sns.heatmap(optimized_flat, ax=axes[1], cmap="viridis", vmin=0, vmax=2*np.pi)
axes[1].set_title("Optimierte Trainingsmatrix")
axes[1].set_xlabel("Parameter")
axes[1].set_ylabel("Qubit")

plt.tight_layout()
plt.savefig(os.path.join(results_dir, "matrix_comparison.png"), dpi=150)
plt.show()

# Berechne die Differenz zwischen den Matrizen
diff_matrix = optimized_flat - original_flat
plt.figure(figsize=(12, 5))
sns.heatmap(diff_matrix, cmap="coolwarm", center=0, vmin=-np.pi, vmax=np.pi)
plt.title("Parameteränderungen durch Optimierung")
plt.xlabel("Parameter")
plt.ylabel("Qubit")
plt.colorbar(label="Differenz (rad)")
plt.tight_layout()
plt.savefig(os.path.join(results_dir, "parameter_changes.png"), dpi=150)
plt.show()

## 12. Validierung der Optimierung

Zum Abschluss validieren wir die Optimierung, indem wir prüfen, ob der Circuit mit der optimierten Trainingsmatrix tatsächlich für jede Eingabematrix den korrekten Zielzustand mit hoher Wahrscheinlichkeit erzeugt.

In [ ]:
# Validiere die Optimierung mit der finalen Trainingsmatrix
validation_results = []

for i, (matrix, name) in enumerate(zip(input_matrices, input_matrix_names)):
    target_state = target_states[name]
    
    # Erstelle einen neuen Circuit mit der optimierten Trainingsmatrix und der aktuellen Eingabematrix
    val_circuit = Circuit(qubits, depth, 
                          training_phases=current_training_matrix.tolist(),
                          activation_phases=matrix.tolist(), 
                          shots=1024)
    
    # Führe den Circuit aus und erhalte die Counts
    val_circuit.run()
    counts = val_circuit.get_counts()
    
    # Berechne die Wahrscheinlichkeiten
    total_shots = sum(counts.values())
    probabilities = {state: count/total_shots for state, count in counts.items()}
    
    # Ermittle den wahrscheinlichsten Zustand
    most_probable_state = max(probabilities.items(), key=lambda x: x[1])
    target_probability = probabilities.get(target_state, 0)
    
    # Speichere die Ergebnisse
    validation_results.append({
        "matrix_name": name,
        "target_state": target_state,
        "most_probable_state": most_probable_state[0],
        "most_probable_prob": most_probable_state[1],
        "target_probability": target_probability,
        "is_correct": most_probable_state[0] == target_state
    })

# Zeige die Validierungsergebnisse
print("Validierungsergebnisse:")
print("-" * 80)
print(f"{'Matrix':15s} | {'Zielzustand':12s} | {'Wahrscheinlichste':15s} | {'Wahrsch.':8s} | {'Korrekt':8s}")
print("-" * 80)

for result in validation_results:
    print(f"{result['matrix_name']:15s} | {result['target_state']:12s} | "
          f"{result['most_probable_state']:15s} | {result['most_probable_prob']:.4f} | "
          f"{('✓' if result['is_correct'] else '✗'):8s}")

print("-" * 80)
print(f"Korrekte Klassifikationen: {sum(1 for r in validation_results if r['is_correct'])} von {len(validation_results)}")

# Visualisiere die Validierungsergebnisse
plt.figure(figsize=(12, 6))

matrix_names = [r["matrix_name"] for r in validation_results]
target_probs = [r["target_probability"] for r in validation_results]
most_probable_probs = [r["most_probable_prob"] for r in validation_results]

x = np.arange(len(matrix_names))
width = 0.35

plt.bar(x - width/2, target_probs, width, label='Zielzustand', color='green')
plt.bar(x + width/2, most_probable_probs, width, label='Wahrscheinlichster Zustand', color='orange')

plt.title("Validierung der Optimierung")
plt.xlabel("Eingabematrix")
plt.ylabel("Wahrscheinlichkeit")
plt.xticks(x, matrix_names)
plt.ylim(0, 1.0)
plt.grid(True, axis='y', alpha=0.3)
plt.legend()

# Füge Textinformationen für jeden Balken hinzu
for i, result in enumerate(validation_results):
    plt.text(i - width/2, result["target_probability"] + 0.02, 
             f"{result['target_probability']:.3f}", ha='center')
    plt.text(i + width/2, result["most_probable_prob"] + 0.02, 
             f"{result['most_probable_prob']:.3f}", ha='center')
    
    # Zeige den Zielzustand unter dem Balken
    plt.text(i, -0.07, f"Ziel: '{result['target_state']}'", ha='center')

plt.tight_layout()
plt.savefig(os.path.join(results_dir, "validation_results.png"), dpi=150)
plt.show()

## 13. Zusammenfassung und Schlussfolgerungen

In diesem Tutorial haben wir gesehen, wie mit dem LLY-DML Framework die Parameter eines Quantum Circuits mit L-Gates optimiert werden können. Wir haben folgende Schritte durchgeführt:

1. **Circuit-Erstellung**: Wir haben einen Circuit mit L-Gates erstellt, der aus einer spezifischen Abfolge von Trainingsphasen, Inputphasen und Hadamard-Gates besteht.

2. **Datenvorbereitung**: Wir haben Trainings- und Eingabematrizen erstellt und jedem Input einen eindeutigen Zielzustand zugeordnet.

3. **Optimierung**: Mit dem Adam-Optimierer haben wir die Trainingsmatrix optimiert, um die Wahrscheinlichkeit zu maximieren, dass der Circuit bei Eingabe einer bestimmten Matrix den zugeordneten Zielzustand liefert.

4. **Validierung**: Wir haben überprüft, ob der optimierte Circuit tatsächlich für die verschiedenen Eingaben die richtigen Zielzustände mit hoher Wahrscheinlichkeit erzeugt.

5. **Visualisierung**: Wir haben die Ergebnisse visualisiert, um ein besseres Verständnis des Optimierungsprozesses zu erhalten.

Dieser Prozess demonstriert die Grundprinzipien des Quantum Machine Learnings, indem die Parameter eines Quantum Circuits trainiert werden, um spezifische Ausgaben für bestimmte Eingaben zu erzeugen - ähnlich zu klassischen neuronalen Netzen, aber mit den einzigartigen Eigenschaften von Quantencomputern.

## 14. Weitere Schritte und Verbesserungen

Hier sind einige mögliche Erweiterungen und Verbesserungen für zukünftige Untersuchungen:

1. **Mehr Optimierer**: Implementieren und vergleichen Sie verschiedene Optimierungsalgorithmen wie SGD, Momentum, RMSprop und Nadam.

2. **Komplexere Daten**: Verwenden Sie reale Datensätze und transformieren Sie diese in geeignete Inputmatrizen.

3. **Hyperparameter-Tuning**: Experimentieren Sie mit verschiedenen Parametern wie Lernrate, Anzahl der Qubits, Tiefe oder Anzahl der Iterationen.

4. **Fehleranalyse**: Untersuchen Sie, warum manche Zielzustände leichter zu optimieren sind als andere.

5. **Circuit-Variationen**: Experimentieren Sie mit verschiedenen Gate-Strukturen und -Kombinationen.

Das LLY-DML Framework bietet eine flexibel erweiterbare Plattform für diese und weitere Experimente im Bereich des Quantum Machine Learnings.